In [ ]:
import  torch, os
import  numpy as np
from    MiniImagenet import MiniImagenet
import  scipy.stats
from    torch.utils.data import DataLoader
from    torch.optim import lr_scheduler
import  random, sys, pickle
import  argparse
import time

#from ANIP import Meta
#from metafgsmanil import Meta
#from metafgsm import Meta
#from MAMLMeta import Meta
#from meta import Meta
#from Adv_Quer import Meta
#from metafgsmnewnew import Meta
from metapgd import Meta

from torch.utils.tensorboard import SummaryWriter
writer = SummaryWriter('../run-imgsz28-4000/pgd-metalr=0.003-advlr=0.0002', comment='pgd-metalr=0.003-advlr=0.0002')

def mean_confidence_interval(accs, confidence=0.95):
    n = accs.shape[0] 
    m, se = np.mean(accs), scipy.stats.sem(accs)
    h = se * scipy.stats.t._ppf((1 + confidence) / 2, n - 1)
    return m, h

def main():
    
    torch.manual_seed(222)
    torch.cuda.manual_seed_all(222)
    np.random.seed(222)

    print(args)
    '''
    config = [
        ('conv2d', [32, 3, 3, 3, 1, 0]),
        ('relu', [True]),
        ('bn', [32]),
        ('max_pool2d', [2, 2, 0]),
        ('conv2d', [32, 32, 3, 3, 1, 0]),
        ('relu', [True]),
        ('bn', [32]),
        ('max_pool2d', [2, 2, 0]),
        ('conv2d', [32, 32, 3, 3, 1, 0]),
        ('relu', [True]),
        ('bn', [32]),
        ('max_pool2d', [2, 2, 0]),
        ('conv2d', [32, 32, 3, 3, 1, 0]),
        ('relu', [True]),
        ('bn', [32]),
        ('max_pool2d', [2, 1, 0]),
        ('flatten', []),
        ('linear', [args.n_way, 32 * 5 * 5])
    ]
    '''
    config = [
        ('conv2d', [32, 3, 3, 3, 1, 0]),
        ('relu', [True]),
        ('bn', [32]),
        ('max_pool2d', [2, 2, 0]),
        ('conv2d', [32, 32, 3, 3, 1, 0]),
        ('relu', [True]),
        ('bn', [32]),
        ('max_pool2d', [2, 2, 0]),
        ('conv2d', [32, 32, 3, 3, 1, 0]),
        ('relu', [True]),
        ('bn', [32]),
        ('max_pool2d', [2, 1, 0]),
        ('flatten', []),
        ('linear', [args.n_way, 32 * 2 * 2])
    ]
    
    device = torch.device('cuda:0')
    maml = Meta(args, config, device).to(device)
    
    
    start_epoch = 0
    start_step = 0
    #filename = 'mamlfgsmeps4_2.pt'
    filename = 'mamlfgsmeps2_8.pt'
    #maml = Meta(args, config).to(device)
    if os.path.isfile(filename):
        print("=> loading checkpoint '{}'".format(filename))
        checkpoint = torch.load(filename)
        start_epoch = checkpoint['epoch']
        start_step = checkpoint['step']
        maml.net.load_state_dict(checkpoint['state_dict'])
        #maml = maml.to(device)
        print("=> loaded checkpoint '{}' (epoch {})"
                  .format(filename, checkpoint['epoch']))
    else:
        print("=> no checkpoint found at '{}'".format(filename))
    

    tmp = filter(lambda x: x.requires_grad, maml.parameters())
    num = sum(map(lambda x: np.prod(x.shape), tmp))
    print(maml)
    print('Total trainable tensors:', num)

    # batchsz here means total episode number
    mini = MiniImagenet('../../dataset/', mode='train', n_way=args.n_way, k_shot=args.k_spt,
                        k_query=args.k_qry,
                        batchsz=4000, resize=args.imgsz)
    mini_test = MiniImagenet('../../dataset/', mode='test', n_way=args.n_way, k_shot=args.k_spt,
                             k_query=args.k_qry,
                             batchsz=100, resize=args.imgsz)
    tot_step = -args.task_num
    #adv_loss_on = True
    for epoch in range(100):
        # fetch meta_batchsz num of episode each time
        db = DataLoader(mini, args.task_num, shuffle=True, num_workers=0, pin_memory=True)
        for step, (x_spt, y_spt, x_qry, y_qry) in enumerate(db): # 0~124 -> batch 32일 때
            tot_step = tot_step + args.task_num
            '''
            if step == 1:
                t = time.perf_counter()
            if step == 499:
                ExecTime = time.perf_counter() - t
                print(ExecTime)
            if step == 501:
                t = time.perf_counter()
            if step == 999:
                ExecTime = time.perf_counter() - t
                print(ExecTime)
            '''
            x_spt, y_spt, x_qry, y_qry = x_spt.to(device), y_spt.to(device), x_qry.to(device), y_qry.to(device)

            accs, accs_adv, loss_q, loss_q_adv = maml(x_spt, y_spt, x_qry, y_qry)

            if step % 10 == 0:
                print('step:', step, '\ttraining acc:', accs)
                print('step:', step, '\ttraining acc_adv:', accs_adv)
                writer.add_scalar("acc/train", accs[-1],tot_step)
                writer.add_scalar("acc_adv/train", accs_adv[-1],tot_step)
                writer.add_scalar("loss/train", loss_q,tot_step)
                writer.add_scalar("loss_adv/train", loss_q_adv,tot_step)
                state = {'epoch': epoch, 'step': step, 'state_dict': maml.net.state_dict()}
                torch.save(state, 'mamlfgsmeps4_2.pt')
            
            if step % 60 == 0:  # evaluation -> 학습에는 전혀 영향을 주지 않음, copy network를 사용하므로
                db_test = DataLoader(mini_test, 1, shuffle=True, num_workers=0, pin_memory=True)
                accs_all_test = []
                accsadv_all_test = []
                accsadvpr_all_test = []
                loss_all_test = []
                loss_adv_all_test = []
                
                for x_spt, y_spt, x_qry, y_qry in db_test:
                    x_spt, y_spt, x_qry, y_qry = x_spt.squeeze(0).to(device), y_spt.squeeze(0).to(device), \
                                                 x_qry.squeeze(0).to(device), y_qry.squeeze(0).to(device)

                    accs, accs_adv, accs_adv_prior, loss_q, loss_q_adv = maml.finetunning(x_spt, y_spt, x_qry, y_qry)
                    accs_all_test.append(accs)
                    accsadv_all_test.append(accs_adv)
                    accsadvpr_all_test.append(accs_adv_prior)
                    loss_all_test.append(loss_q.item())
                    loss_adv_all_test.append(loss_q_adv.item())
                    
                # [b, update_step+1]
                accs = np.array(accs_all_test).mean(axis=0).astype(np.float16)
                accs_adv = np.array(accsadv_all_test).mean(axis=0).astype(np.float16)
                accs_adv_prior = np.array(accsadvpr_all_test).mean(axis=0).astype(np.float16)
                loss_q = np.array(loss_all_test).mean()
                loss_q_adv = np.array(loss_adv_all_test).mean()
                print('Test acc:', accs)
                print('Test acc_adv:', accs_adv)
                print('Test acc_adv_prior:', accs_adv_prior)
                writer.add_scalar("acc/test", accs[-1],tot_step)
                writer.add_scalar("acc_adv/test", accs_adv[-1],tot_step)
                writer.add_scalar("acc_adv_prior/test", accs_adv_prior[-1],tot_step)
                writer.add_scalar("loss/test", loss_q,tot_step)
                writer.add_scalar("loss_adv/test", loss_q_adv,tot_step)
                if step==120:
                    writer.add_scalar("acc/test_epoch", accs[-1],epoch)
                    writer.add_scalar("acc_adv/test_epoch", accs_adv[-1],epoch)
                    writer.add_scalar("loss/epoch", loss_q, epoch)
                    writer.add_scalar("loss_adv/epoch", loss_q_adv, epoch)
            

if __name__ == '__main__':

    argparser = argparse.ArgumentParser()
    argparser.add_argument('--epoch', type=int, help='epoch number', default=60000)
    argparser.add_argument('--n_way', type=int, help='n way', default=5)
    argparser.add_argument('--k_spt', type=int, help='k shot for support set', default=1)
    argparser.add_argument('--k_qry', type=int, help='k shot for query set', default=15)
    argparser.add_argument('--imgsz', type=int, help='imgsz', default=28)
    argparser.add_argument('--imgc', type=int, help='imgc', default=3)
    argparser.add_argument('--task_num', type=int, help='meta batch size, namely task num', default=32)
    argparser.add_argument('--meta_lr', type=float, help='meta-level outer learning rate', default=0.001) #0.001 - 0.0002 기존
    argparser.add_argument('--update_lr', type=float, help='task-level inner update learning rate', default=0.01)
    argparser.add_argument('--update_step', type=int, help='task-level inner update steps', default=5)
    argparser.add_argument('--update_step_test', type=int, help='update steps for finetunning', default=10)
    
    #argparser.add_argument('--fast', action="store_true", help='whether to use fgsm')

    args = argparser.parse_args(args=[])

    main()

Namespace(epoch=60000, imgc=3, imgsz=28, k_qry=15, k_spt=1, meta_lr=0.001, n_way=5, task_num=32, update_lr=0.01, update_step=5, update_step_test=10)
init
=> no checkpoint found at 'mamlfgsmeps2_8.pt'
Meta(
  (net): Learner(
    conv2d:(ch_in:3, ch_out:32, k:3x3, stride:1, padding:0)
    relu:(True,)
    bn:(32,)
    max_pool2d:(k:2, stride:2, padding:0)
    conv2d:(ch_in:32, ch_out:32, k:3x3, stride:1, padding:0)
    relu:(True,)
    bn:(32,)
    max_pool2d:(k:2, stride:2, padding:0)
    conv2d:(ch_in:32, ch_out:32, k:3x3, stride:1, padding:0)
    relu:(True,)
    bn:(32,)
    max_pool2d:(k:2, stride:1, padding:0)
    flatten:()
    linear:(in:128, out:5)
    
    (vars): ParameterList(
        (0): Parameter containing: [torch.float32 of size 32x3x3x3 (GPU 0)]
        (1): Parameter containing: [torch.float32 of size 32 (GPU 0)]
        (2): Parameter containing: [torch.float32 of size 32 (GPU 0)]
        (3): Parameter containing: [torch.float32 of size 32 (GPU 0)]
        (4): Param

step: 120 	training acc: [0.19625    0.28166667 0.305      0.3125     0.31875    0.32291667]
step: 120 	training acc_adv: [0.         0.         0.         0.         0.         0.11166667]
Test acc: [0.1969 0.2316 0.255  0.26   0.261  0.2612 0.2637 0.2615 0.2625 0.2617
 0.262 ]
Test acc_adv: [0.02347 0.05588 0.0825  0.104   0.1152  0.1266  0.1343  0.1401  0.1437
 0.1458  0.1492 ]
Test acc_adv_prior: [0.1194 0.2396 0.318  0.3962 0.4358 0.4778 0.504  0.5327 0.5444 0.5527
 0.566 ]
step: 0 	training acc: [0.20375    0.29666667 0.32291667 0.33666667 0.3375     0.33458333]
step: 0 	training acc_adv: [0.     0.     0.     0.     0.     0.1175]
Test acc: [0.2006 0.2483 0.261  0.2668 0.2688 0.2715 0.2727 0.2742 0.2734 0.2722
 0.2734]
Test acc_adv: [0.02347 0.0544  0.0807  0.1036  0.1173  0.1273  0.1328  0.1405  0.1447
 0.1488  0.1527 ]
Test acc_adv_prior: [0.115  0.218  0.3044 0.382  0.4329 0.464  0.4827 0.509  0.525  0.542
 0.555 ]
step: 10 	training acc: [0.19583333 0.26541667 0.28791667 0.3

step: 60 	training acc: [0.18583333 0.29333333 0.32708333 0.33625    0.33708333 0.34166667]
step: 60 	training acc_adv: [0.         0.         0.         0.         0.         0.15541667]
Test acc: [0.1868 0.2313 0.2603 0.2693 0.2734 0.276  0.2737 0.2744 0.2764 0.2764
 0.2769]
Test acc_adv: [0.02907 0.06744 0.1064  0.1315  0.1464  0.1583  0.1635  0.1709  0.1754
 0.1794  0.1823 ]
Test acc_adv_prior: [0.1462 0.2878 0.408  0.4873 0.5366 0.571  0.597  0.62   0.6313 0.646
 0.6567]
step: 70 	training acc: [0.20041667 0.25875    0.27791667 0.27666667 0.27791667 0.2775    ]
step: 70 	training acc_adv: [0.         0.         0.         0.         0.         0.11333333]
step: 80 	training acc: [0.21       0.30625    0.32333333 0.33166667 0.3325     0.33166667]
step: 80 	training acc_adv: [0.         0.         0.         0.         0.         0.15958333]
step: 90 	training acc: [0.21291667 0.315      0.3325     0.33166667 0.33375    0.33541667]
step: 90 	training acc_adv: [0.         0.         

step: 0 	training acc: [0.19375    0.3025     0.32541667 0.33041667 0.34041667 0.34166667]
step: 0 	training acc_adv: [0.         0.         0.         0.         0.         0.17708333]
Test acc: [0.2007 0.2568 0.2751 0.277  0.2769 0.2793 0.2793 0.281  0.281  0.2827
 0.2854]
Test acc_adv: [0.04706 0.08624 0.1305  0.1594  0.1708  0.1768  0.1838  0.187   0.1891
 0.1914  0.192  ]
Test acc_adv_prior: [0.2244 0.334  0.4712 0.5713 0.612  0.6255 0.652  0.661  0.6685 0.6743
 0.6714]
step: 10 	training acc: [0.18208333 0.30916667 0.33291667 0.34791667 0.34875    0.35083333]
step: 10 	training acc_adv: [0.         0.         0.         0.         0.         0.17541667]
step: 20 	training acc: [0.1975     0.29375    0.32583333 0.34166667 0.33958333 0.34125   ]
step: 20 	training acc_adv: [0.         0.         0.         0.         0.         0.15833333]
step: 30 	training acc: [0.20375    0.29583333 0.33041667 0.35041667 0.35458333 0.35833333]
step: 30 	training acc_adv: [0.         0.         0

step: 70 	training acc: [0.20375    0.3075     0.32416667 0.335      0.33375    0.33666667]
step: 70 	training acc_adv: [0.         0.         0.         0.         0.         0.16791667]
step: 80 	training acc: [0.18958333 0.32583333 0.36291667 0.37666667 0.37666667 0.37583333]
step: 80 	training acc_adv: [0.      0.      0.      0.      0.      0.19625]
step: 90 	training acc: [0.21333333 0.31333333 0.33833333 0.34666667 0.34833333 0.35083333]
step: 90 	training acc_adv: [0.         0.         0.         0.         0.         0.16583333]
step: 100 	training acc: [0.2225     0.31875    0.33333333 0.33666667 0.34208333 0.34375   ]
step: 100 	training acc_adv: [0.         0.         0.         0.         0.         0.15708333]
step: 110 	training acc: [0.19083333 0.32333333 0.34583333 0.3475     0.35083333 0.34875   ]
step: 110 	training acc_adv: [0.         0.         0.         0.         0.         0.16541667]
step: 120 	training acc: [0.195      0.3125     0.34666667 0.34666667 0.34

step: 10 	training acc: [0.205      0.36708333 0.39208333 0.3975     0.39625    0.39541667]
step: 10 	training acc_adv: [0.         0.         0.         0.         0.         0.18791667]
step: 20 	training acc: [0.18458333 0.31708333 0.34583333 0.35125    0.35291667 0.35208333]
step: 20 	training acc_adv: [0.         0.         0.         0.         0.         0.17083333]
step: 30 	training acc: [0.20458333 0.32541667 0.35375    0.3675     0.36458333 0.36083333]
step: 30 	training acc_adv: [0.      0.      0.      0.      0.      0.18875]
step: 40 	training acc: [0.18958333 0.32833333 0.36541667 0.37583333 0.38291667 0.38666667]
step: 40 	training acc_adv: [0.         0.         0.         0.         0.         0.18833333]
step: 50 	training acc: [0.21041667 0.32708333 0.35083333 0.3575     0.35875    0.35833333]
step: 50 	training acc_adv: [0.         0.         0.         0.         0.         0.18458333]
step: 60 	training acc: [0.19708333 0.31291667 0.34708333 0.35583333 0.3608333